In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path

2023-10-17 11:02:12.481261: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("train*.tfrecords"))

In [3]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['age']
    return ecg_data, label

def drop_na_ages(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

def age_lt_90(x,y):
    return tf.math.reduce_all(tf.math.less_equal(y, tf.constant([90.0])))

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_ages)
    dataset = dataset.filter(age_lt_90)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

2023-10-17 11:02:18.660518: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46582 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [6]:
def conv_block(x, n, k, mp, groups = 12):
    x = tf.keras.layers.Conv1D(n * groups, k, padding='same', groups = groups)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    if mp:
        x = tf.keras.layers.MaxPooling1D(mp)(x)

    return x

input_layer = tf.keras.layers.Input(shape=(5000, 12))
x = input_layer

x = conv_block(x, 16, 7, 2)
x = conv_block(x, 16, 5, 4)
x = conv_block(x, 32, 5, 2)
x = conv_block(x, 32, 5, 4)
x = conv_block(x, 64, 5, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)

x = tf.keras.layers.Conv1D(128, 12, padding = 'same', data_format = 'channels_first')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling1D()(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1)(x)

model = tf.keras.models.Model(input_layer, x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)

In [7]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

In [8]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(make_checkpoint_dir("data/models", "cnn-age"))
]

model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10


2023-10-17 11:02:24.836632: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5585e4c656e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-17 11:02:24.836684: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2023-10-17 11:02:24.969510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-10-17 11:02:26.392739: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-10-17 11:02:28.656230: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-17 11:02:28.894386: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.

   7479/Unknown - 773s 101ms/step - loss: 444.3855 - mse: 444.3855 - mae: 15.2362INFO:tensorflow:Assets written to: data/models/cnn-age-2023-10-17_11-02-20/assets


INFO:tensorflow:Assets written to: data/models/cnn-age-2023-10-17_11-02-20/assets


7479/7479 [==============================] - 907s 119ms/step - loss: 444.3855 - mse: 444.3855 - mae: 15.2362 - val_loss: 139.6660 - val_mse: 139.6660 - val_mae: 9.5143 - lr: 0.0010
Epoch 2/10
7479/7479 [==============================] - ETA: 0s - loss: 187.7240 - mse: 187.7240 - mae: 10.9002INFO:tensorflow:Assets written to: data/models/cnn-age-2023-10-17_11-02-20/assets


...

7479/7479 [==============================] - 881s 118ms/step - loss: 134.3425 - mse: 134.3425 - mae: 9.2107 - val_loss: 111.0324 - val_mse: 111.0324 - val_mae: 8.3506 - lr: 0.0010
Epoch 9/10
7479/7479 [==============================] - ETA: 0s - loss: 128.8280 - mse: 128.8280 - mae: 9.0186INFO:tensorflow:Assets written to: data/models/cnn-age-2023-10-17_11-02-20/assets


INFO:tensorflow:Assets written to: data/models/cnn-age-2023-10-17_11-02-20/assets


7479/7479 [==============================] - 880s 118ms/step - loss: 128.8280 - mse: 128.8280 - mae: 9.0186 - val_loss: 116.6657 - val_mse: 116.6657 - val_mae: 8.4377 - lr: 0.0010
Epoch 10/10
7479/7479 [==============================] - ETA: 0s - loss: 123.7718 - mse: 123.7718 - mae: 8.8392INFO:tensorflow:Assets written to: data/models/cnn-age-2023-10-17_11-02-20/assets


INFO:tensorflow:Assets written to: data/models/cnn-age-2023-10-17_11-02-20/assets


7479/7479 [==============================] - 885s 118ms/step - loss: 123.7718 - mse: 123.7718 - mae: 8.8392 - val_loss: 124.8845 - val_mse: 124.8845 - val_mae: 8.7287 - lr: 0.0010


In [9]:
model.save("data/models/cnn-age.keras")